In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('PATIENTS.csv')
df.head()

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,9467,10006,F,2094-03-05 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,2165-08-12 00:00:00,1
1,9472,10011,F,2090-06-05 00:00:00,2126-08-28 00:00:00,2126-08-28 00:00:00,NaN,1
2,9474,10013,F,2038-09-03 00:00:00,2125-10-07 00:00:00,2125-10-07 00:00:00,2125-10-07 00:00:00,1
3,9478,10017,F,2075-09-21 00:00:00,2152-09-12 00:00:00,NaN,2152-09-12 00:00:00,1
4,9479,10019,M,2114-06-20 00:00:00,2163-05-15 00:00:00,2163-05-15 00:00:00,2163-05-15 00:00:00,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   row_id       100 non-null    int64 
 1   subject_id   100 non-null    int64 
 2   gender       100 non-null    object
 3   dob          100 non-null    object
 4   dod          100 non-null    object
 5   dod_hosp     70 non-null     object
 6   dod_ssn      77 non-null     object
 7   expire_flag  100 non-null    int64 
dtypes: int64(3), object(5)
memory usage: 6.4+ KB


In [6]:
df.isnull().sum()

row_id          0
subject_id      0
gender          0
dob             0
dod             0
dod_hosp       30
dod_ssn        23
expire_flag     0
dtype: int64

In [7]:
df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
df['dod'] = pd.to_datetime(df['dod'], errors='coerce')
df['dod_hosp'] = pd.to_datetime(df['dod_hosp'], errors='coerce')
df['dod_ssn'] = pd.to_datetime(df['dod_ssn'], errors='coerce')
df['gender'] = df['gender'].astype('category')

In [8]:
df = df.drop_duplicates()

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   row_id       100 non-null    int64         
 1   subject_id   100 non-null    int64         
 2   gender       100 non-null    category      
 3   dob          100 non-null    datetime64[ns]
 4   dod          100 non-null    datetime64[ns]
 5   dod_hosp     70 non-null     datetime64[ns]
 6   dod_ssn      77 non-null     datetime64[ns]
 7   expire_flag  100 non-null    int64         
dtypes: category(1), datetime64[ns](4), int64(3)
memory usage: 5.8 KB


In [10]:
df.head()

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,9467,10006,F,2094-03-05,2165-08-12,2165-08-12,2165-08-12,1
1,9472,10011,F,2090-06-05,2126-08-28,2126-08-28,NaT,1
2,9474,10013,F,2038-09-03,2125-10-07,2125-10-07,2125-10-07,1
3,9478,10017,F,2075-09-21,2152-09-12,NaT,2152-09-12,1
4,9479,10019,M,2114-06-20,2163-05-15,2163-05-15,2163-05-15,1


In [15]:
df.to_csv("patients_cleaned.csv", index=False)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   row_id       100 non-null    int64         
 1   subject_id   100 non-null    int64         
 2   gender       100 non-null    category      
 3   dob          100 non-null    datetime64[ns]
 4   dod          100 non-null    datetime64[ns]
 5   dod_hosp     70 non-null     datetime64[ns]
 6   dod_ssn      77 non-null     datetime64[ns]
 7   expire_flag  100 non-null    int64         
dtypes: category(1), datetime64[ns](4), int64(3)
memory usage: 5.8 KB


In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq

# Path to CSV
csv_path = "patients_cleaned.csv"

# Download data
df_cleaned = pd.read_csv(csv_path)

# Convert columns to datetime
date_cols = ['dob', 'dod', 'dod_hosp', 'dod_ssn']
for col in date_cols:
    df_cleaned[col] = pd.to_datetime(df_cleaned[col], errors='coerce')

# Manually define schema in a Hive-compatible manner
schema = pa.schema([
    pa.field('row_id', pa.int64()),           # Hive: bigint
    pa.field('subject_id', pa.int64()),       # Hive: bigint
    pa.field('gender', pa.string()),          # Hive: string
    pa.field('dob', pa.timestamp('ns')),      # Hive: timestamp
    pa.field('dod', pa.timestamp('ns')),      # Hive: timestamp
    pa.field('dod_hosp', pa.timestamp('ns')), # Hive: timestamp
    pa.field('dod_ssn', pa.timestamp('ns')),  # Hive: timestamp
    pa.field('expire_flag', pa.int32())       # Hive: int
])

# Convert the DataFrame to a PyArrow table and apply the schema
table = pa.Table.from_pandas(df_cleaned)
table = table.cast(schema)

# Save the file in Parquet format compatible with Hive
parquet_path = "patients_cleaned.parquet"
pq.write_table(
    table,
    parquet_path,
    version='2.6',
    use_dictionary=True,
    compression='SNAPPY',
    flavor='spark'
)